In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Preprocessing
import h5py
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin

from tensorflow import keras

In [ ]:
def load_dataset():
    train_dataset = h5py.File('/content/drive/MyDrive/course 21-22/data science u-tad/DL/train_signs.h5', "r")
    X_train = np.array(train_dataset["train_set_x"][:])
    y_train = np.array(train_dataset["train_set_y"][:])

    test_dataset = h5py.File('/content/drive/MyDrive/course 21-22/data science u-tad/DL/test_signs.h5', "r")
    X_test = np.array(test_dataset["test_set_x"][:])
    y_test = np.array(test_dataset["test_set_y"][:])

    classes = np.array(test_dataset["list_classes"][:])
    
    return X_train, y_train, X_test, y_test, classes

In [ ]:
X_train, y_train, X_test, y_test, classes = load_dataset()

In [ ]:
# One Hot Encoding target variable
le = LabelBinarizer().fit(y_train)
y_train_label = le.transform(y_train)
y_test_label = le.transform(y_test)

# Escala
X_train = X_train / 255.
X_test = X_test / 255.

# Modelo desarrollado desde cero

In [ ]:
# Preparación del modelo
model = keras.models.Sequential()
# Convolución 32 filtros
model.add(keras.layers.Conv2D(32,kernel_size=(3,3),padding='same',input_shape=[64, 64, 3], use_bias=False))
# Aplicación Batch Normalization
model.add(keras.layers.BatchNormalization())
# Aplicación Función de activación relu
model.add(keras.layers.Activation("relu"))
# Aplicación MaxPooling
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

# Convolución 16 fitros
model.add(keras.layers.Conv2D(16,kernel_size=(3,3),padding='same',input_shape=[16, 16, 3]))
# Aplicación Batch Normalization
model.add(keras.layers.BatchNormalization())
# Aplicación Función de activación relu
model.add(keras.layers.Activation("relu"))

# Aplicación Avg Pooling
model.add(keras.layers.AveragePooling2D(pool_size=(2,2)))
# Convertir las entradas en un vector para poder pasarlo a una capa Dense
model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(512, activation="relu"))
model.add(keras.layers.Dense(128, activation="relu"))
# Crear la capa de salida para clasificación con 6 posibles clases mutuamente excluyentes
model.add(keras.layers.Dense(6, activation="softmax"))

# Entrenamiento y validación del modelo
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train_label,epochs=20,batch_size=32)
model.evaluate(X_test, y_test_label, batch_size=32)


Epoch 1/20
34/34 [==============================] - 11s 295ms/step - loss: 2.1169 - accuracy: 0.3787
Epoch 2/20
34/34 [==============================] - 9s 265ms/step - loss: 0.5924 - accuracy: 0.7759
Epoch 3/20
34/34 [==============================] - 10s 287ms/step - loss: 0.2381 - accuracy: 0.9213
Epoch 4/20
34/34 [==============================] - 9s 270ms/step - loss: 0.1162 - accuracy: 0.9648
Epoch 5/20
34/34 [==============================] - 9s 263ms/step - loss: 0.0897 - accuracy: 0.9722
Epoch 6/20
34/34 [==============================] - 9s 263ms/step - loss: 0.0414 - accuracy: 0.9907
Epoch 7/20
34/34 [==============================] - 9s 264ms/step - loss: 0.0195 - accuracy: 0.9991
Epoch 8/20
34/34 [==============================] - 9s 265ms/step - loss: 0.0079 - accuracy: 1.0000
Epoch 9/20
34/34 [==============================] - 9s 264ms/step - loss: 0.0049 - accuracy: 1.0000
Epoch 10/20
34/34 [==============================] - 9s 263ms/step - loss: 0.0036 - accuracy: 1.00

[0.06095738708972931, 0.9750000238418579]

# Modelo con uso de transfer learning

Carga de una red ResNet.

In [ ]:
model = keras.applications.resnet50.ResNet50(weights="imagenet")

102981632/102967424 [==============================] - 3s 0us/step


In [ ]:
#Importación de modelo base (Xception) sin las capas de clasificación
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)
# Aplicación de Global Average Pooling
avg = keras.layers.GlobalAveragePooling2D()(base_model.output) 
# Creación de capa de salida para clasificación con 6 mutuamente excluyentes
output = keras.layers.Dense(6, activation="softmax")(avg) 
#Definición del modelo
model = keras.Model(inputs=base_model.input, outputs=output)

83697664/83683744 [==============================] - 1s 0us/step


In [ ]:
# Congelación de los pesos del modelo base para sacar el máximo partido del modelo preentrenado
base_model.trainable = False
    
# Comenzar con una constante de aprendizaje un poco más agresiva
optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)

# Entrenamiento y evaluación del modelo
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train_label,epochs=20,batch_size=32)
model.evaluate(X_test, y_test_label, batch_size=32)

Epoch 1/20
34/34 [==============================] - 19s 479ms/step - loss: 1.4422 - accuracy: 0.4769
Epoch 2/20
34/34 [==============================] - 16s 475ms/step - loss: 1.1059 - accuracy: 0.6907
Epoch 3/20
34/34 [==============================] - 16s 475ms/step - loss: 0.9587 - accuracy: 0.7417
Epoch 4/20
34/34 [==============================] - 16s 476ms/step - loss: 0.8603 - accuracy: 0.7824
Epoch 5/20
34/34 [==============================] - 16s 475ms/step - loss: 0.7874 - accuracy: 0.8148
Epoch 6/20
34/34 [==============================] - 16s 478ms/step - loss: 0.7304 - accuracy: 0.8380
Epoch 7/20
34/34 [==============================] - 16s 477ms/step - loss: 0.6854 - accuracy: 0.8361
Epoch 8/20
34/34 [==============================] - 16s 477ms/step - loss: 0.6455 - accuracy: 0.8537
Epoch 9/20
34/34 [==============================] - 16s 478ms/step - loss: 0.6106 - accuracy: 0.8602
Epoch 10/20
34/34 [==============================] - 16s 479ms/step - loss: 0.5793 - accura

[0.4453344941139221, 0.8916666507720947]

In [ ]:
# Congelación de los pesos del modelo base para sacar el máximo partido del modelo preentrenado
base_model.trainable = False
    
# Ajuste de constante de aprendizaje
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, decay=0.0001)

# Entrenamiento y evaluación del modelo
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train_label,epochs=20,batch_size=32)
model.evaluate(X_test, y_test_label, batch_size=32)

Epoch 1/20
34/34 [==============================] - 19s 485ms/step - loss: 0.3904 - accuracy: 0.9278
Epoch 2/20
34/34 [==============================] - 16s 481ms/step - loss: 0.3670 - accuracy: 0.9315
Epoch 3/20
34/34 [==============================] - 16s 479ms/step - loss: 0.3514 - accuracy: 0.9370
Epoch 4/20
34/34 [==============================] - 16s 479ms/step - loss: 0.3348 - accuracy: 0.9380
Epoch 5/20
34/34 [==============================] - 16s 477ms/step - loss: 0.3228 - accuracy: 0.9444
Epoch 6/20
34/34 [==============================] - 16s 478ms/step - loss: 0.3092 - accuracy: 0.9481
Epoch 7/20
34/34 [==============================] - 16s 481ms/step - loss: 0.2984 - accuracy: 0.9528
Epoch 8/20
34/34 [==============================] - 16s 484ms/step - loss: 0.2886 - accuracy: 0.9546
Epoch 9/20
34/34 [==============================] - 16s 485ms/step - loss: 0.2804 - accuracy: 0.9574
Epoch 10/20
34/34 [==============================] - 16s 479ms/step - loss: 0.2711 - accura

[0.30118435621261597, 0.9166666865348816]

El modelo óptimo es el desarrollado desde cero ya que la precisión es mayor.